<a href="https://colab.research.google.com/github/ezeguins/Bounding-Box-Detecction-VGG16/blob/main/model_training_ts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

THE NOTEBOOKS PERFORMS THE FOLLOWING:
- Load the dataset generated at "dataset_generator_ts.ipynb" file
- Initialize model parameters
- partition the data into training and testing splits using 90% of the data for training and the remaining 10% for testing
- Load the VGG16 network with "imagenet" weights, ensuring the head FC layers are left off
- Construct the model we will fine-tune for bounding box regression
- Initialize and train de model
- Plot results and model training history

In [ ]:
import os
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
version = 20
detector = 'detector' + str(version) + '.h5'
plot = 'plot' + str(version) + '.png'
test_images = 'test_images' + str(version) + '.txt'

BASE_PATH = "/content/gdrive/MyDrive/VGG16/dataset"
IMAGES_PATH = os.path.sep.join([BASE_PATH, "images"])
ANNOTS_PATH = os.path.sep.join([BASE_PATH, "etiquetas_export-data-60_solobloques.csv"])

# define the path to the base output directory
BASE_OUTPUT = "/content/gdrive/MyDrive/VGG16/output"
# define the path to the output serialized model, model training plot,
# and testing image filenames
MODEL_PATH = os.path.sep.join([BASE_OUTPUT, detector])
PLOT_PATH = os.path.sep.join([BASE_OUTPUT, plot])
TEST_FILENAMES = os.path.sep.join([BASE_OUTPUT, test_images])
print(MODEL_PATH)

/content/gdrive/MyDrive/VGG16/output/detector20.h5


In [ ]:
# initialize our initial learning rate, number of epochs to train
# for, and the batch size
INIT_LR = 1e-4
NUM_EPOCHS = 25
BATCH_SIZE = 100

  TRAIN.PY

In [ ]:
# import the necessary packages
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [ ]:
outfile = "/content/gdrive/MyDrive/VGG16/dataset/datos_full.npz"
npzfile = np.load(outfile)
data1 = npzfile['arr_0']
targets1 = npzfile['arr_1']
filenames1 = npzfile['arr_2']

In [ ]:

# partition the data into training and testing splits using 90% of
# the data for training and the remaining 10% for testing
split = train_test_split(data1, targets1, filenames1, test_size=0.10,	random_state=42)
# unpack the data split
(trainImages, testImages) = split[:2]
(trainTargets, testTargets) = split[2:4]
(trainFilenames, testFilenames) = split[4:]
# write the testing filenames to disk so that we can use then
# when evaluating/testing our bounding box regressor
print("[INFO] saving testing filenames...")
f = open(TEST_FILENAMES, "w")
f.write("\n".join(testFilenames))
f.close()

[INFO] saving testing filenames...


In [ ]:
from tensorflow.keras.layers import Dropout

In [ ]:
# load the VGG16 network, ensuring the head FC layers are left off
vgg = VGG16(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))
# freeze all VGG layers so they will *not* be updated during the
# training process
for layer in vgg.layers[:15]:
    layer.trainable = False

# flatten the max-pooling output of VGG
flatten = vgg.output
flatten = Flatten()(flatten)

# construct a fully-connected layer header to output the predicted
# bounding box coordinates

bboxHead = Dense(128, activation="relu")(flatten)
bboxHead = Dropout(0.2)(bboxHead) # Dropout layer to reduce overfitting
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
bboxHead = Dense(4, activation="sigmoid")(bboxHead)
# construct the model we will fine-tune for bounding box regression
model = Model(inputs=vgg.input, outputs=bboxHead)
for i, layer in enumerate(model.layers):
    print(i, layer.name, layer.trainable)

from keras.callbacks import ReduceLROnPlateau
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.6, patience=4, verbose=1, mode='max', min_lr=0)

In [ ]:
# initialize the optimizer, compile the model, and show the model
# summary
opt = Adam(lr=INIT_LR)
model.compile(loss="mse", optimizer=opt)
print(model.summary())
# train the network for bounding box regression
print("[INFO] training bounding box regressor...")
H = model.fit(trainImages, trainTargets,validation_data=(testImages, testTargets),
	batch_size=BATCH_SIZE,
	epochs=NUM_EPOCHS,
	verbose=1, callbacks=[lr_reduce])

In [ ]:
print("[INFO] saving object detector model...")
model.save(MODEL_PATH, save_format="h5")
# plot the model training history
N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.title("Bounding Box Regression Loss on Training Set")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="lower left")
plt.savefig(PLOT_PATH)